In [3]:
# Dependencies
import requests
import json
import pandas as pd

from pprint import pprint

# Google developer API key
import sys
sys.path.append('../..')
from config import gkey
from config import owm_api


file = pd.read_csv("winedata_edited.csv")

file.head(30)

,Unnamed: 0,Title,Variety,Year,Price,Grade,Winery,Province,Region,Country,Taster,Taster_Twitter,Description
0,1,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,2011,15,87,Quinta dos Avidagos,Douro,NaN,Portugal,Roger Voss,@vossroger,"This is ripe and fruity, a wine that is smooth..."
1,2,Rainstorm 2013 Pinot Gris (Willamette Valley),Pinot Gris,2013,14,87,Rainstorm,Oregon,Willamette Valley,US,Paul Gregutt,@paulgwine,"Tart and snappy, the flavors of lime flesh and..."
2,3,St. Julian 2013 Reserve Late Harvest Riesling ...,Riesling,2013,13,87,St. Julian,Michigan,Lake Michigan Shore,US,Alexander Peartree,NaN,"Pineapple rind, lemon pith and orange blossom ..."
3,4,Sweet Cheeks 2012 Vintner's Reserve Wild Child...,Pinot Noir,2012,65,87,Sweet Cheeks,Oregon,Willamette Valley,US,Paul Gregutt,@paulgwine,"Much like the regular bottling from 2012, this..."
4,5,Tandem 2011 Ars In Vitro Tempranillo-Merlot (N...,Tempranillo-Merlot,2011,15,87,Tandem,Northern Spain,Navarra,Spain,Michael Schachner,@wineschach,Blackberry and raspberry aromas show a typical...
5,6,Terre di Giurfo 2013 Belsito Frappato (Vittoria),Frappato,2013,16,87,Terre di Giurfo,Sicily & Sardinia,Vittoria,Italy,Kerin O’Keefe,@kerinokeefe,"Here's a bright, informal red that opens with ..."
6,7,Trimbach 2012 Gewurztraminer (Alsace),Gewürztraminer,2012,24,87,Trimbach,Alsace,Alsace,France,Roger Voss,@vossroger,This dry and restrained wine offers spice in p...
7,8,Heinz Eifel 2013 Shine Gewürztraminer (Rheinhe...,Gewürztraminer,2013,12,87,Heinz Eifel,Rheinhessen,NaN,Germany,Anna Lee C. Iijima,NaN,Savory dried thyme notes accent sunnier flavor...
8,9,Jean-Baptiste Adam 2012 Les Natures Pinot Gris...,Pinot Gris,2012,27,87,Jean-Baptiste Adam,Alsace,Alsace,France,Roger Voss,@vossroger,This has great depth of flavor with its fresh ...
9,10,Kirkland Signature 2011 Mountain Cuvée Caberne...,Cabernet Sauvignon,2011,19,87,Kirkland Signature,California,Napa Valley,US,Virginie Boone,@vboone,"Soft, supple plum envelopes an oaky structure ..."


In [18]:
file = file.loc[file["Province"] != "America"]

In [21]:
province = []

for p in file["Province"]:
    if p not in province:
        province.append(p)

In [23]:
# Target location

province_response =[]

# Run API calls for each winery in list
for prov in province:

    target_location = prov
    
    # Build the endpoint URL
    target_url = ('https://maps.googleapis.com/maps/api/geocode/json?'
    'address={0}&key={1}').format(target_location, gkey)
    
    # Run a request to endpoint and convert result to json
    geo_data = requests.get(target_url).json()
   
    # Add response to our list
    province_response.append(geo_data)        

In [25]:
# Collect and store latitude and longitude for each province

lat = []
lng = []

i=0

# Loop through wineries and append coordinate lists
for response in province_response:
    try:
        latitude = province_response[i]["results"][0]["geometry"]["location"]["lat"]
        longitude = province_response[i]["results"][0]["geometry"]["location"]["lng"]
        
        lat.append(latitude)
        lng.append(longitude)
        
        i = i+1
    except:
        lat.append("lat error")
        lng.append("lng error")
    
        i = i+1

In [29]:
province_df= pd.DataFrame({"Province" : province,
                          "Latitude" : lat,
                          "Longitude" : lng})
province_df

,Province,Latitude,Longitude
0,Douro,41.512,-5.49225
1,Oregon,43.8041,-120.554
2,Michigan,44.3148,-85.6024
3,Northern Spain,43.2636,-2.92907
4,Sicily & Sardinia,37.6,14.0154
5,Alsace,48.3182,7.44162
6,Rheinhessen,43.1909,-79.3006
7,California,36.7783,-119.418
8,Mosel,43.8056,-87.7404
9,Other,40.7245,-73.9979


In [32]:
# Drop rows without location data
province_df = province_df.loc[province_df["Latitude"] != "lat error"]
province_df

,Province,Latitude,Longitude
0,Douro,41.512,-5.49225
1,Oregon,43.8041,-120.554
2,Michigan,44.3148,-85.6024
3,Northern Spain,43.2636,-2.92907
4,Sicily & Sardinia,37.6,14.0154
5,Alsace,48.3182,7.44162
6,Rheinhessen,43.1909,-79.3006
7,California,36.7783,-119.418
8,Mosel,43.8056,-87.7404
9,Other,40.7245,-73.9979


In [36]:
# Add coordinates to primary dataframe by merging
complete_df = pd.merge(file, province_df, on="Province")

complete_df

,Unnamed: 0,Title,Variety,Year,Price,Grade,Winery,Province,Region,Country,Taster,Taster_Twitter,Description,Latitude,Longitude
0,1,Quinta dos Avidagos 2011 Avidagos Red (Douro),Portuguese Red,2011,15,87,Quinta dos Avidagos,Douro,NaN,Portugal,Roger Voss,@vossroger,"This is ripe and fruity, a wine that is smooth...",41.512,-5.49225
1,608,Quinta do Portal 2012 Verdelho and Sauvignon B...,Portuguese White,2012,15,87,Quinta do Portal,Douro,NaN,Portugal,Roger Voss,@vossroger,Verdelho and Sauvignon Blanc hangs together ea...,41.512,-5.49225
2,647,Quinta do Monte Xisto 2013 Red (Douro),Portuguese Red,2013,80,92,Quinta do Monte Xisto,Douro,NaN,Portugal,Roger Voss,@vossroger,"This is an impressive, powerful wine. It speak...",41.512,-5.49225
3,784,Quinta do Sagrado 2008 VT '08 Red (Douro),Portuguese Red,2008,32,90,Quinta do Sagrado,Douro,NaN,Portugal,Roger Voss,@vossroger,"This mature wine has a soft, smooth character ...",41.512,-5.49225
4,786,Sobredos 2012 Aneto Tinto Red (Douro),Portuguese Red,2012,12,90,Sobredos,Douro,NaN,Portugal,Roger Voss,@vossroger,"A ripe and structured wine, this has bold blac...",41.512,-5.49225
5,874,Quinta do Vallado 2009 Sousão (Douro),Sousão,2009,40,90,Quinta do Vallado,Douro,NaN,Portugal,Roger Voss,@vossroger,"Sousão gives intense color and that, along wit...",41.512,-5.49225
6,887,Maritávora 2009 Reserva Branco White (Douro),Portuguese White,2009,60,90,Maritávora,Douro,NaN,Portugal,Roger Voss,@vossroger,Aromas of fresh-cut hay lead to a wine that ha...,41.512,-5.49225
7,901,Quinta do Vallado 2010 Reserva Branco White (D...,Portuguese White,2010,50,90,Quinta do Vallado,Douro,NaN,Portugal,Roger Voss,@vossroger,"A wood-aged white, spicy and ripe. It has just...",41.512,-5.49225
8,947,Real Companhia Velha 2014 Evel Tinto Red (Douro),Portuguese Red,2014,13,87,Real Companhia Velha,Douro,NaN,Portugal,Roger Voss,@vossroger,Just lightly touched by wood this is a bright ...,41.512,-5.49225
9,1072,Poças 2014 Símbolo Red (Douro),Portuguese Red,2014,150,94,Poças,Douro,NaN,Portugal,Roger Voss,@vossroger,This is a major Douro wine from the Poças fami...,41.512,-5.49225


In [41]:
type(complete_df)

pandas.core.frame.DataFrame

In [44]:
complete_df.to_csv('winedata_coord.csv')

In [ ]:
import gmaps
import numpy as np


# Configure gmaps with API key
gmaps.configure(api_key=gkey)


# Store 'Lat' and 'Lng' into  locations 
locations = file[["latitude", "longitude"]].astype(float)

In [ ]:
# Create winery symbol layer
winery_layer = gmaps.marker_layer(
    locations
)


fig = gmaps.figure()
fig.add_layer(winery_layer)

fig

In [10]:
from config import weather_api

weather_url = "http://api.worldweatheronline.com/premium/v1/past-weather.ashx?q=" 

In [ ]:
type(just_file[["longitude", "latitude"]])
winery_coords = file[["longitude", "latitude"]]
type(winery_coords.iloc[1,1])

In [ ]:
latitude_float = [float(x) for x in winery_coords["latitude"]]

winery_coords["latitude"] = latitude_float

type(winery_coords.iloc[1,1])

In [ ]:
# Northern hemisphere growing season weather lists
jul = []
aug = []
sep = []

# Southern hemisphere growing season weather lists
jan = []
feb = []
mar = []

# Loop through wineries and append weather lists


for index, row in winery_coords.iterrows():
#    try:
        if row["latitude"] < 0:
            latit = (row["latitude"])
            longi = (row["longitude"])
        
 # Run a request to endpoint and convert result to json
            jul_weather = requests.get(f"{weather_url}{lat},{lng}&key={weather_api}&date=2011-07-01&enddate=2011-07-31&tp=24&format=json").json()
            aug_weather = requests.get(f"{weather_url}{lat},{lng}&key={weather_api}&date=2011-08-01&enddate=2011-08-31&tp=24&format=json").json()
            sep_weather = requests.get(f"{weather_url}{lat},{lng}&key={weather_api}&date=2011-09-01&enddate=2011-09-30&tp=24&format=json").json()
                
            jul.append(jul_weather)
            aug.append(aug_weather)
            sep.append(sep_weather)
            
        else:
#            try:
                latit = (row["latitude"])
                longi = (row["longitude"])

                jan_weather = requests.get(f"{weather_url}{lat},{lng}&key={weather_api}&date=2011-01-01&enddate=2011-01-31&tp=24&format=json").json()
                feb_weather = requests.get(f"{weather_url}{lat},{lng}&key={weather_api}&date=2011-02-01&enddate=2011-02-28&tp=24&format=json").json()
                mar_weather = requests.get(f"{weather_url}{lat},{lng}&key={weather_api}&date=2011-03-01&enddate=2011-03-31&tp=24&format=json").json()

                jan.append(jul_weather)
                feb.append(aug_weather)
                mar.append(sep_weather)

               
       

In [11]:

import gmaps
import numpy as np

test = requests.get(f"{weather_url}38.936866,-77.038317&key={weather_api}&date=2019-01-01&enddate=2019-01-15&tp=24&format=json").json()
pprint(test)

{'data': {'request': [{'query': 'Lat 38.94 and Lon -77.04', 'type': 'LatLon'}],
          'weather': [{'astronomy': [{'moon_illumination': '34',
                                      'moon_phase': 'Waning Crescent',
                                      'moonrise': '03:09 AM',
                                      'moonset': '02:03 PM',
                                      'sunrise': '07:27 AM',
                                      'sunset': '04:57 PM'}],
                       'date': '2019-01-01',
                       'hourly': [{'DewPointC': '7',
                                   'DewPointF': '44',
                                   'FeelsLikeC': '11',
                                   'FeelsLikeF': '52',
                                   'HeatIndexC': '12',
                                   'HeatIndexF': '53',
                                   'WindChillC': '11',
                                   'WindChillF': '52',
                                   'WindGustKmph': '29',

In [26]:
test["data"]["request"]

[{'type': 'LatLon', 'query': 'Lat 38.94 and Lon -77.04'}]

In [40]:
testweather = test["data"]["weather"]
testweather

[{'date': '2019-01-01',
  'astronomy': [{'sunrise': '07:27 AM',
    'sunset': '04:57 PM',
    'moonrise': '03:09 AM',
    'moonset': '02:03 PM',
    'moon_phase': 'Waning Crescent',
    'moon_illumination': '34'}],
  'maxtempC': '13',
  'maxtempF': '56',
  'mintempC': '11',
  'mintempF': '52',
  'totalSnow_cm': '0.0',
  'sunHour': '0.0',
  'uvIndex': '0',
  'hourly': [{'time': '24',
    'tempC': '13',
    'tempF': '56',
    'windspeedMiles': '8',
    'windspeedKmph': '13',
    'winddirDegree': '303',
    'winddir16Point': 'WNW',
    'weatherCode': '119',
    'weatherIconUrl': [{'value': 'http://cdn.worldweatheronline.net/images/wsymbols01_png_64/wsymbol_0003_white_cloud.png'}],
    'weatherDesc': [{'value': 'Cloudy'}],
    'precipMM': '2.0',
    'humidity': '72',
    'visibility': '10',
    'pressure': '1015',
    'cloudcover': '42',
    'HeatIndexC': '12',
    'HeatIndexF': '53',
    'DewPointC': '7',
    'DewPointF': '44',
    'WindChillC': '11',
    'WindChillF': '52',
    'WindGust

In [50]:
fahrenheit = []

x = 0
for w in testweather:
    fahrenheit.append(testweather[x]["hourly"][0]["tempF"])
    x += 1

    tempdf = pd.DataFrame([fahrenheit])
    tempdf.T.mean()

precip = []
x = 0
for p in testweather:
    precip.append(testweather[x]["hourly"][0]["precipMM"])
    x += 1

    precipdf = pd.DataFrame([precip])
    precipdf.T.mean()

humidity = []
x = 0
for h in testweather:
    humidity.append(testweather[x]["hourly"][0]["humidity"])
    x += 1
    precipdf.T.mean()

['72',
 '70',
 '69',
 '73',
 '70',
 '63',
 '63',
 '77',
 '55',
 '54',
 '55',
 '60',
 '73',
 '65',
 '71']

In [ ]:
locations = region[["lat", "lng"]]

# Plot Heatmap
fig = gmaps.figure()

# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=avg_temps, 
                                 dissipating=False, max_intensity=10,
                                 point_radius=1)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig